In [ ]:
from snowflake.snowpark import Session, udf
from snowflake.snowpark.functions import *
from config import snowpark_config
import os

if snowpark_config:
    session = Session.builder.configs(snowpark_config).create()
else:
    connection_parameters = {
        "account": os.environ["snowflake_account"],
        "user": os.environ["snowflake_user"],
        "password": os.environ["snowflake_password"],
        "role": os.environ["snowflake_user_role"],
        "warehouse": os.environ["snowflake_warehouse"],
        "database": os.environ["snowflake_database"],
        "schema": os.environ["snowflake_schema"]
    }
    session = Session.builder.configs(connection_parameters).create()

In [ ]:
articles_df = session.table('articles')

In [ ]:
articles_df = articles_df.select(col('author'), col('claps'), col('title'), col('reading_time'))

In [ ]:
# standardize claps
replacedClaps = when(col('claps').like('%K'), regexp_replace(col('claps'), 'K', '').cast('DOUBLE')*1000).otherwise(col('claps').cast('DOUBLE'))
df = articles_df.select('author', replacedClaps.cast('INTEGER').alias('claps'), 'title', 'reading_time')

In [ ]:
# get the article with the most claps for each author
max_df = df.groupBy('author').agg(max('claps').alias('claps'))
# join back in the title and reading time, and remove duplicates
joined = max_df.join(df, ['author', 'claps']).distinct()

In [161]:
# update table
joined.write.save_as_table('top_articles_by_author', mode='overwrite')